In [3]:
#import general libraries
import os
import time as tm
import numpy as np

#import serial to be able to directly communicate with teensy if desired
import serial as ser

#import camera library
import PySpin

#import modules to make acquiring and saving video easier
import cv2 as cv

import sys as sys
sys.path.insert(0,'/Users/michaelreitman/github/cloned_repositories/arcadia-phenotypomat/phenotyper')
sys.path.insert(0,'utils/')
import cam_tools as ct
import vidtools as vt


In [4]:
#define arduino port to connect to
arduino_name ='/dev/cu.usbmodem149606801' #'/dev/cu.usbmodem144168401

#initialize camera variables
frame_rate = 90 #aquisition frame rate in hz
exposure_time = 5000 #exposure time in us
gain = 10 #multiplicative gain (AU)
length = 120 #recording length in seconds

pixel_bin_size = 4 #number of pixels to bin in x and y (1 means no binning)

#define path and name for output videos
output_path = '../Data/Test Videos/20240129/'
sample = 'VWRcoverglass_lightsoff_'
prefix = str(length) + 's_' + str(frame_rate) + 'Hz_' + str(exposure_time) + 'us_' + str(pixel_bin_size) + 'xbin_'+str(sample)
barcode = '00'

#define codec to use (if any)
fourcc_code = 0 #cv.VideoWriter_fourcc(*'mp4v') #0 means no compression use Videowriter_fourcc(*'') to look up extension codes
ext = '.avi' #append appropriate string to end of video name


In [1]:
#test lights work

dev=ser.Serial(arduino_name) # establish serial communication with microcontroller

dev.write(b'SET ALTERNATE_STATUS 0;') # Make sure alternate mode is off or direct LED control won't work

dev.write(b'SET BLUE_STATUS 1;')
tm.sleep(0.5)

dev.write(b'SET BLUE_STATUS 0;') # turn off the LED

dev.write(b'SET VIOLET_STATUS 1;')
tm.sleep(0.5)

dev.write(b'SET VIOLET_STATUS 0;') # turn off the LED


NameError: name 'ser' is not defined

In [7]:
#start camera
if not ct.detect_cams():  # make sure there are blackfly cameras attached to the computer
 sys.exit()

system = PySpin.System.GetInstance() #start the communication to the cameras

cam_list=system.GetCameras() #get the handles of the cameras

cam=cam_list[0] #set a handle for a specific camera

cam.Init() #initialze the camera

#make sure the camera isn't streaming or some fields won't be writable
try:
    cam.EndAcquisition()
except:
    None

#get camera node map
node_map = cam.GetTLStreamNodeMap()

#set gain
cam.GainAuto.SetValue(PySpin.GainAuto_Off)
cam.Gain.SetValue(gain)

#set exposure
cam.ExposureAuto.SetValue(PySpin.ExposureAuto_Off) # turn off auto exposure
cam.ExposureTime.SetValue(exposure_time) #set the exposure time on the camera

#turn off automatic grey value
cam.AutoExposureTargetGreyValueAuto.SetValue(False)

#turn off automatic gamma correction
cam.GammaEnable.SetValue(False)

#turn off auto sharpness
genable = PySpin.CBooleanPtr(node_map.GetNode("SharpenessAuto"))
if PySpin.IsWritable(genable):
    genable.SetValue(0)

#set framerate
cam.AcquisitionFrameRateEnable.SetValue(True)

if frame_rate > cam.AcquisitionFrameRate.GetMax(): #make sure it doesn't exceed the max
    frame_rate = cam.AcquisitionFrameRate.GetMax()
    print('frame rate exceeds max. Setting to: ' + str(frame_rate))

ct.set_framerate(cam,frame_rate)

#bin pixels and reset FOV afterwards to avoid artifacts
cam.BinningHorizontal.SetValue(pixel_bin_size)
cam.BinningVertical.SetValue(pixel_bin_size)

cam.OffsetX.SetValue(0)
cam.OffsetY.SetValue(0)
cam.Height.SetValue(cam.Height.GetMax())
cam.Width.SetValue(cam.Width.GetMax())

#set output line
cam.LineSelector.SetValue(PySpin.LineSelector_Line1)
#cam.V3_3Enable.SetValue(False) # If using line 2 turn off 3.3V pullup resistor so the line can be used for output
#cam.LineMode.SetValue(PySpin.LineMode_Output) #if using line 2 set to output mode
cam.LineSource.SetValue(PySpin.LineSource_ExposureActive) #route desired output to Line 1 (try Counter0Active or ExposureActive)

#turn on chunk mode to get camera meta data
cam.ChunkModeActive.SetValue(True)
try:
    chunk_has_line_status = True
    cam.ChunkSelector.SetValue(PySpin.ChunkSelector_ExposureEndLineStatusAll)
    cam.ChunkEnable.SetValue(True)
except:
    chunk_has_line_status = False
try:
    cam.ChunkSelector.SetValue(PySpin.ChunkSelector_FrameID)
    cam.ChunkEnable.SetValue(True)
except:
    pass
    cam.ChunkSelector.SetValue(PySpin.ChunkSelector_Timestamp)
    cam.ChunkEnable.SetValue(True)


In [ ]:
#determine max data that can be held in memory
#I put this in because I was having kernel crashing issues but this doesn't seem to be happening anymore
#so I'm commenting it out for now MR 2023-01-18

#determine the size of one image
# try:
#     cam.BeginAcquisition()
# except:
#     None
# image = cam.GetNextImage()
# image_size = image.GetImageSize()
# cam.AcquisitionStop()

# data_per_sec =image_size*frame_rate #calculate image data per second

# print("{:e}".format(data_per_sec) + ' pixels per second')

# max_memory =0.9*32*10**9 #only use up at most 90% of available memory when acquiring
# print("{:e}".format(max_memory) + " {}% of RAM".format(90))

# max_length =max_memory/data_per_sec #calculate how many seconds to reach 90% RAM usage
# print(str(max_length) + 's max recording time')

# #change the recording length if it exceeds the max that can be held in memory
# if length>max_length:
#     length=max_length
#     prefix = str(length) + 's_' + str(frame_rate) + 'Hz_' + str(exposure_time) + 'us_' + str(sample)
#     print('requested video length exceeds memory storage, recording length set to ' + str(length) + ' seconds')


In [5]:
#preview recording
dev=ser.Serial(arduino_name) # establish serial communication with microcontroller
dev.write(b'SET ALTERNATE_STATUS 1;') # set up for alternation
app = vt.CameraApp(cam)
app.run()
app.close()


In [ ]:
#playing with buffer settings
NUM_BUFFERS = 10

# set the buffer to read olderst first
handling_mode = PySpin.CEnumerationPtr(node_map.GetNode('StreamBufferHandlingMode'))
handling_mode_entry = handling_mode.GetEntryByName('OldestFirst')
handling_mode.SetIntValue(handling_mode_entry.GetValue())

#set the number of buffers (set to max if user defined exceeds max)
buffer_count = PySpin.CIntegerPtr(node_map.GetNode('StreamBufferCountManual'))
maxbuffers = PySpin.CIntegerPtr(node_map.GetNode('StreamBufferCountMax'))
buffer_count.SetValue(int(np.min([NUM_BUFFERS,maxbuffers.GetValue()])))

buffer_mode = PySpin.CEnumerationPtr(node_map.GetNode('StreamBufferCountMode'))
buffer_mode.SetIntValue(PySpin.StreamBufferCountMode_Manual)

print(buffer_count.GetValue())

#check we can see dropped frames
dropped_frame = PySpin.CIntegerPtr(node_map.GetNode('StreamDroppedFrameCount'))
print(dropped_frame.GetValue())


In [8]:
#test recording with alternating LEDs
dev=ser.Serial(arduino_name) # establish serial communication with microcontroller

#acquire blue
dev.write(b'SET ALTERNATE_STATUS 1;') # set up for alternation

images,host_timestamps,dropped_frames,frame_ids,active_sequence,cam_timestamps = vt.grab_images(cam, node_map,length=length) #get video images and metadata

dev.write(b'SET ALTERNATE_STATUS 0;') # turn off alternation

vt.save_video(images,path=output_path, #save the images to a video
               fourcc_code=fourcc_code, ext=ext,
               prefix=prefix,
               barcode=barcode,frame_rate=frame_rate)


np.savetxt(os.path.join(output_path,prefix+'_'+barcode+'_metadata.csv'),
            np.transpose([host_timestamps,dropped_frames,frame_ids,active_sequence,cam_timestamps]),
            header ='host_timestamps, dropped_frames,frame_ids,active_sequence,cam_timestamps',
              delimiter=',')


In [ ]:
#test recording with blue LED fixed on
dev=ser.Serial(arduino_name) # establish serial communication with microcontroller

#acquire blue
dev.write(b'SET BLUE_STATUS 1;') # turn on the led

tm.sleep(2)

images,timestamps = ct.grab_images(cam, length=length) #get images and timestamps for a vide of 10 seconds length from the camera
dev.write(b'SET BLUE_STATUS 0;') #turn off the led

print(str(timestamps[-1]-timestamps[0])) #print the total imaging time
print(str(timestamps[-1]-timestamps[-2])) #print the time between frames
ct.save_avi(images, frame_rate=frame_rate, barcode =barcode, prefix=prefix,path=output_path) #save the images to a video


In [ ]:
#test recording with violet LED fixed on

#set length in seconds
length = 10

#set file names
prefix =  '10s_30Hz_nematodes_20231219'
barcode = '00'

#acquire violet
dev.write(b'SET VIOLET_STATUS 1;') # turn on the led

tm.sleep(2)

images,timestamps = ct.grab_images(cam, length=length) #get images and timestamps for a vide of 10 seconds length from the camera
dev.write(b'SET VIOLET_STATUS 0;') #turn off the led

print(str(timestamps[-1]-timestamps[0])) #print the total imaging time
print(str(timestamps[-1]-timestamps[-2])) #print the time between frames

ct.save_avi(images, frame_rate=frame_rate, barcode =barcode, prefix=prefix,path=output_path) #save the images to a video

#turn off camera strobe voltage
cam.LineSource.SetValue(PySpin.LineSource_UserOutput0) #route desired output to Line 1 (try Counter0Active or ExposureActive)
cam.UserOutputSelector.SetValue(False)

del cam #turn off the camera

cam_list.Clear() # de-initialize the cameras

system.ReleaseInstance() #release the handlers


In [ ]:
## stop the camera from sending voltage out if need be
cam.LineSelector.SetValue(PySpin.LineSelector_Line2)
cam.LineSource.SetValue(PySpin.LineSource_UserOutput0) #route desired output to Line 1 (try Counter0Active or ExposureActive)
cam.UserOutputSelector.SetValue(False)


In [ ]:
cam.AcquisitionStop() #turn off the camera

del cam #turn off the camera

cam_list.Clear() # de-initialize the cameras

system.ReleaseInstance() #release the handlers
